In [2]:
# Imports
import pandas as pd
import re

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [8]:
url ='https://api.census.gov/data/timeseries/poverty/saipe/schdist?get=SD_NAME,SAEPOVALL_PT&for=school%20district%20(elementary):*&time=2010&key=90d62fa12906f0cca6e3b8587c62fe7e5ad81675'

In [9]:
res=requests.get(url)

### Step 1: Create a soup object from the home page

In [10]:
soup = BeautifulSoup(res.content, 'lxml')

In [15]:
soup

<html><body><p>[["SD_NAME","SAEPOVALL_PT","time","state","school district (elementary)"],
["Clarkdale-Jerome Elementary District","3789","2010","04","00004"],
["Salome Consolidated Elementary District","2759","2010","04","00005"],
["Vernon Elementary District","1936","2010","04","00022"],
["Aguila Elementary District","1182","2010","04","00480"],
["Alhambra Elementary District","84718","2010","04","00600"],
["Alpine Elementary District","464","2010","04","00630"],
["Apache Elementary District","167","2010","04","00750"],
["Arlington Elementary District","1476","2010","04","00840"],
["Ash Creek Elementary District","706","2010","04","00870"],
["Avondale Elementary District","44779","2010","04","00960"],
["Balsz Elementary District","29114","2010","04","01050"],
["Beaver Creek Elementary District","4809","2010","04","01080"],
["Blue Elementary District","37","2010","04","01230"],
["Bonita Elementary District","1193","2010","04","01260"],
["Bouse Elementary District","1179","2010","04","0

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and path from the `href`. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'href': 'restaurants/1.html'}, 
    {'name': "Applebee's", 'href': 'restaurants/2.html'},
    ...
]
```

In [6]:
table = soup.find('table', {'id': 'heisman'})

In [7]:
restaurants = []
for row in table.find('tbody').find_all('tr'):
    store = {}
    store['name'] = row.find('a').text.strip()
    store['href'] = row.find('a').attrs['href'].replace('mailto:', '')
    restaurants.append(store)


In [ ]:
restaurants[0:5]

### Step 3: Using the `href`, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [69]:
foods = []
restaurants = []

for row in table.find('tbody').find_all('tr'):
    stuff = {}
    stuff['restaurant'] = row.find('a').text.strip()
    url = f'https://pages.git.generalassemb.ly/rldaggie/for-scraping/{row.find("a")["href"]}'
    res=requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    table2 = soup.find('table')
    for row in table2.find('tbody').find_all('tr'):
        stuff['calories'] = row.find_all('td')[0].text
        stuff['carbs'] = row.find_all('td')[1].text
        stuff['category'] = row.find_all('td')[2].text
        stuff['fat'] = row.find_all('td')[3].text
        stuff['name'] = row.find_all('td')[4].text
        foods.append(stuff)
            
#             url = f'https://pages.git.generalassemb.ly/rldaggie/for-scraping/{row.find("a")["href"]}'

In [70]:
foods[0:5]

[{'restaurant': 'A&W Restaurants',
  'calories': 'A&W® Root Beer',
  'carbs': 'Drinks',
  'category': '270',
  'fat': '0',
  'name': '72'},
 {'restaurant': 'A&W Restaurants',
  'calories': 'A&W® Root Beer',
  'carbs': 'Drinks',
  'category': '270',
  'fat': '0',
  'name': '72'},
 {'restaurant': 'A&W Restaurants',
  'calories': 'A&W® Root Beer',
  'carbs': 'Drinks',
  'category': '270',
  'fat': '0',
  'name': '72'},
 {'restaurant': 'A&W Restaurants',
  'calories': 'A&W® Root Beer',
  'carbs': 'Drinks',
  'category': '270',
  'fat': '0',
  'name': '72'},
 {'restaurant': 'A&W Restaurants',
  'calories': 'A&W® Root Beer',
  'carbs': 'Drinks',
  'category': '270',
  'fat': '0',
  'name': '72'}]

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 5,131 rows

In [72]:
test = pd.DataFrame(foods)

### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [74]:
test.to_csv('./datasets/foods.csv', index=False)